- https://medium.com/@kartha.kishan/solving-openai-carracing-v0-using-image-processing-5e1005ee0cb
- https://github.com/KishanKartha/Solving-OpenAI-CarRacing-v0/blob/main/Solving_OpenAI_CarRacing_v0_using_PID_controllers_.ipynb

In [12]:
import gymnasium as gym
from gymnasium.wrappers import monitoring
import glob
import io
import base64
from IPython.display import HTML
from pyvirtualdisplay import Display
from IPython import display as ipythondisplay
import cv2
import numpy as np 
import matplotlib.pyplot as plt 

In [14]:
display = Display(visible=0, size=(1400, 900))
display.start()

"""
Utility functions to enable video recording of gym environment 
and displaying it.
To enable video, just do "env = wrap_env(env)""
"""

def show_video():
  mp4list = glob.glob('video/*.mp4')
  if len(mp4list) > 0:
    mp4 = mp4list[0]
    video = io.open(mp4, 'r+b').read()
    encoded = base64.b64encode(video)
    ipythondisplay.display(HTML(data=''''''.format(encoded.decode('ascii'))))
  else: 
    print("Could not find video")
    


FileNotFoundError: [WinError 2] 지정된 파일을 찾을 수 없습니다

In [18]:

def wrap_env(env):
  env = monitoring(env, './video', force=True)
  return env

In [19]:
def find_error(observation,previous_error):

  def green_mask(observation):
    hsv = cv2.cvtColor(observation, cv2.COLOR_BGR2HSV)
    mask_green = cv2.inRange(hsv, (36, 25, 25), (70, 255,255))

    ## slice the green
    imask_green = mask_green>0
    green = np.zeros_like(observation, np.uint8)
    green[imask_green] = observation[imask_green]
    return(green)


  def gray_scale(observation):
    gray = cv2.cvtColor(observation, cv2.COLOR_RGB2GRAY)
    return gray


  def blur_image(observation):
    blur = cv2.GaussianBlur(observation, (5, 5), 0)
    return blur


  def canny_edge_detector(observation):
    canny = cv2.Canny(observation, 50, 150)
    return canny


  cropped = observation[63:65, 24:73]


  green = green_mask(cropped)
  grey  = gray_scale(green)
  blur  = blur_image(grey)
  canny = canny_edge_detector(blur)

  #find all non zero values in the cropped strip.
  #These non zero points(white pixels) corresponds to the edges of the road
  nz = cv2.findNonZero(canny)
  print('nz', nz)

  #horizontal cordinates of center of the road in the cropped slice
  mid  = 24
  
  # some further adjustments obtained through trail and error
  if nz[:,0,0].max() == nz[:,0,0].min():
    if nz[:,0,0].max() <30 and nz[:,0,0].max()>20:
      return previous_error
    if nz[:,0,0].max() >= mid:
      return(-15)
    else:
      return(+15)
  else:
    return(((nz[:,0,0].max() + nz[:,0,0].min())/2)-mid)


In [20]:
def pid(error,previous_error):
    Kp = 0.02
    Ki = 0.03
    Kd = 0.2   

    steering = Kp * error + Ki * (error + previous_error) + Kd * (error - previous_error)

    return steering

In [30]:
# import gym

# env = wrap_env(gym.make('CarRacing-v2'))
env = gym.make('CarRacing-v2',
            render_mode = 'state_pixels', 
            # continuous = False
            )

observation = env.reset()
env.render() 
rewardsum = 0  
previous_error = 0    


for i,  x in enumerate([1,0]*500):      
    try:
      error = find_error(observation,previous_error)
    except:
      error = -15
      print("error")
      pass

    print('error', error, previous_error)
    steering = pid(error,previous_error)
    print(i, 'steering', steering)
   
    action = (steering,x,0)
    print(i, action)

    observation, reward, terminated, truncated, info = env.step(action)
    previous_error =error
    rewardsum = rewardsum +reward
    print(i, 'rewardsum', rewardsum)

    done = (terminated or truncated)
    if done :
      env.close()
      break
    
print("reward", rewardsum)
# show_video()

error
error -15 0
0 steering -3.75
0 (-3.75, 1, 0)
0 rewardsum 7.120216606498195
error -3.5 -15
1 steering 1.6750000000000003
1 (1.6750000000000003, 0, 0)
1 rewardsum 7.020216606498195
error -5.0 -3.5
2 steering -0.655
2 (-0.655, 1, 0)
2 rewardsum 6.920216606498196
error -6.5 -5.0
3 steering -0.775
3 (-0.775, 0, 0)
3 rewardsum 6.820216606498196
error -7.5 -6.5
4 steering -0.77
4 (-0.77, 1, 0)
4 rewardsum 6.720216606498196
error -8.5 -7.5
5 steering -0.8500000000000001
5 (-0.8500000000000001, 0, 0)
5 rewardsum 6.620216606498197
error -10.0 -8.5
6 steering -1.055
6 (-1.055, 1, 0)
6 rewardsum 6.520216606498197
error -9.5 -10.0
7 steering -0.6749999999999999
7 (-0.6749999999999999, 0, 0)
7 rewardsum 6.420216606498197
error -10.5 -9.5
8 steering -1.01
8 (-1.01, 1, 0)
8 rewardsum 6.320216606498198
error 0.0 -10.5
9 steering 1.7850000000000001
9 (1.7850000000000001, 0, 0)
9 rewardsum 6.220216606498198
error 0.5 0.0
10 steering 0.125
10 (0.125, 1, 0)
10 rewardsum 6.120216606498198
error 0.0 0.